<a href="https://colab.research.google.com/github/Agatsyadav2003/quantized-llm-mobile/blob/main/Perplexity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary dependencies
!pip install -q sentencepiece cmake

# Clone the llama.cpp repository
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!mkdir build && cd build && cmake -DGGML_CUDA=on .. && make -j
%cd ..

Cloning into 'llama.cpp'...
remote: Enumerating objects: 49519, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 49519 (delta 123), reused 72 (delta 70), pack-reused 49354 (from 3)
Receiving objects: 100% (49519/49519), 103.31 MiB | 16.19 MiB/s, done.
Resolving deltas: 100% (35696/35696), done.
/content/llama.cpp
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Tes

In [ ]:
!ls /content/llama.cpp/build/bin/

libggml-base.so		       llama-quantize
libggml-cpu.so		       llama-qwen2vl-cli
libggml-cuda.so		       llama-retrieval
libggml.so		       llama-run
libllama.so		       llama-save-load-state
libllava_shared.so	       llama-server
libmtmd_shared.so	       llama-simple
llama-batched		       llama-simple-chat
llama-batched-bench	       llama-speculative
llama-bench		       llama-speculative-simple
llama-cli		       llama-tokenize
llama-convert-llama2c-to-ggml  llama-tts
llama-cvector-generator        llama-vdot
llama-embedding		       test-arg-parser
llama-eval-callback	       test-autorelease
llama-export-lora	       test-backend-ops
llama-gemma3-cli	       test-barrier
llama-gen-docs		       test-c
llama-gguf		       test-chat
llama-gguf-hash		       test-chat-template
llama-gguf-split	       test-gbnf-validator
llama-gritlm		       test-gguf
llama-imatrix		       test-grammar-integration
llama-infill		       test-grammar-parser
llama-llava-cli		       test-json-schema-to-grammar
llama-

In [ ]:
# Move the uploaded file into your llama.cpp folder
!mv /content/llama_3.2_3b_q4_k_m.gguf /content/llama.cpp/

In [ ]:
# Step 4: Create examples directory in the correct location and download the dataset
!mkdir -p /content/llama.cpp/examples
!pip install -q datasets

# Use Python to fetch & write out the raw test set
from datasets import load_dataset
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

with open("/content/llama.cpp/examples/wikitext-2-raw.txt", "w", encoding="utf-8") as f:
    for line in ds["text"]:
        f.write(line.rstrip() + "\n")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
!cd /content/llama.cpp/build/bin && \
  ./llama-perplexity \
    -m ../../llama_3.2_3b_q4_k_m.gguf \
    -f ../../examples/wikitext-2-raw.txt \
    -ngl 33 \
    -t 4

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA L4, compute capability 8.9, VMM: yes
build: 5215 (5f5e39e1) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
llama_model_load_from_file_impl: using device CUDA0 (NVIDIA L4) - 22503 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 255 tensors from ../../llama_3.2_3b_q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_m

In [ ]:
# Install required Python libs
!pip install -q datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00


In [ ]:
# Install libraries from Hugging Face’s fast pip server
!pip install -q --index-url https://download.pytorch.org/whl/cpu datasets evaluate


In [ ]:
import subprocess
from datasets import load_dataset
import evaluate

# Load a small subset of CNN/DailyMail
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:20]")

predictions = []
references  = []

for example in dataset:
    prompt = f"Summarize the following article:\n{example['article']}\nSummary:"
    cmd = [
        "/content/llama.cpp/build/bin/llama-run",
        "-t", "4",
        "--ngl", "33",
        "/content/llama.cpp/llama_3.2_3b_q4_k_m.gguf",
        prompt
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    summary = result.stdout.strip()

    predictions.append(summary)
    references.append([example['highlights']])

    print("=== Example ===")
    print("REF:", example['highlights'])
    print("PRED:", summary)
    print("------\n")

# Compute BLEU
bleu = evaluate.load("bleu")
bleu_result = bleu.compute(predictions=predictions, references=references)
print(f"\nFinal BLEU score: {bleu_result['bleu']*100:.2f}")


=== Example ===
REF: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
PRED: The Assistant Editor will be responsible for the day-to-day operation of the website. The position will work with the Editor to ensure the website is up and running on time. The position will be responsible for the following duties:
1.	To provide technical support to the Editorial team
2.	To support the website editor in managing the content for the website and social media
3.	To manage and maintain the CMS system
4.	To be responsible for all technical issues related to the website
5.	To perform other related duties as assigned
Skills:
1.	Minimum 2 years of experience in editing websites
2.	Minimum 2 years of experience in website development
3.	Minimum 2 years of experience in handling CMS system
4.	Minimum 2 years of experience